In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!pip install -U keras
!pip install -U tensorflow

In [ ]:
import tensorflow 
from  tensorflow.keras.models import Model
from  tensorflow.keras.layers import Input, LSTM, Dense
from  tensorflow.keras.utils import *
from  tensorflow.keras.initializers import *
from  tensorflow.keras.optimizers import Adam # - Works
from  keras.models import load_model


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# df = pd.read_json('m23.json')
df = pd.read_json('/content/m23.json')

In [ ]:
df

,id,original_text,segmented_text,equation,ans
0,1,Bryan took a look at his books as well . If Br...,Bryan took a look at his books as well . If Br...,x=56*9,504.0
1,2,"For the fifth grade play , the chairs have bee...","For the fifth grade play , the chairs have bee...",X=(27.0*16.0),432.0
2,3,There are 41 short trees and 44 tall trees cur...,There are 41 short trees and 44 tall trees cur...,X=41+57,98.0
3,4,Conner has 25000 dollars in his bank account ....,Conner has 25000 dollars in his bank account ....,x=25000.0-(1500.0*8.0),13000.0
4,5,There are 34 dogwood trees currently in the pa...,There are 34 dogwood trees currently in the pa...,X=34+49,83.0
...,...,...,...,...,...
2368,2369,A plane travels due East at 325 miles per hour...,A plane travels due East at 325 miles per hour...,x=2100.0/(325.0+275.0),3.5
2369,2370,In fourth grade there were 35 students at the ...,In fourth grade there were 35 students at the ...,X=((35.0+10.0)-10.0),35.0
2370,2371,A number increased by 15 is equal to 2.5 the n...,A number increased by 15 is equal to 2.5 the n...,x=15.0/(2.5-1),10.0
2371,2372,"In Haley ’s class , 5 are boys who love to pla...","In Haley ’s class , 5 are boys who love to pla...",x=35/5,7.0


In [ ]:
batch_size = 64
latent_dim = 256
num_samples = 1000

In [ ]:
#Vectorize the data.
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

In [ ]:
input_texts = df['original_text'].tolist()
target_texts = df['equation'].tolist()


In [ ]:
for i in range(len(target_texts)):
  target_texts[i]= '\t' + target_texts[i] + '\n'


In [ ]:
len(input_texts)

2373

In [ ]:
for i in range(len(input_texts)):#check if correct
  
  input_text = input_texts[i]
  target_text = target_texts[i]
  for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
  for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

In [ ]:
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2373
Number of unique input tokens: 75
Number of unique output tokens: 33
Max sequence length for inputs: 469
Max sequence length for outputs: 68


In [ ]:
#Define data for encoder and decoder
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_in_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_in_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

#Define and process the input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Using `encoder_states` set up the decoder as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#Final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
#Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 75)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 33)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        339968      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
#Model data Shape
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_in_data shape: (2373, 469, 75)
decoder_in_data shape: (2373, 68, 33)
decoder_target_data shape: (2373, 68, 33)


In [ ]:
#Compiling and training the model
model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')

model.fit([encoder_in_data, decoder_in_data], decoder_target_data, batch_size = batch_size, epochs=50, validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
 2/30 [=>............................] - ETA: 1:11 - loss: 0.8590

KeyboardInterrupt: ignored

In [ ]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())

#Define Decode Sequence
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Get the first character of target sequence with the start character.
    target_seq[0, 0, target_token_id['\t']] = 1.

    #Sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Exit condition: either hit max length
        #or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

In [ ]:
model.save('model_1.h5')


NameError: ignored

In [ ]:
loadedModel = tf.keras.Models.load_model("/content/model_1.h5")

AttributeError: ignored

In [ ]:
from keras.models import load_model

In [ ]:
model11 = load_model('model_1.h5') # YAY!